In [1]:
from lightfm import LightFM


D:\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from pprint import pprint
from collections import defaultdict

DATA_PATH = 'D:\download'
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    })

ser = pd.read_csv("WTforserials(num3).csv")
mm = pd.read_csv("WTformnmov.csv")
OLD = pd.read_csv("WTforfilms(evr0204).csv")
rate=pd.read_csv('ratingsappend(notintransactions).csv')



OLD=OLD.append(ser)
OLD=OLD.append(rate)
OLD=OLD.append(mm)

In [3]:
pd.set_option('display.max_columns', 50)

from collections import Counter
catal=pd.read_json(os.path.join(DATA_PATH, 'catalogue.json'), orient='index') 
catal['element_uid']=catal.index

catal=catal.reset_index()
catal=catal.drop(['index'], 1)

In [4]:
catal.availability=catal.availability.astype(str)
filmlist=catal[catal.availability == '[]'].element_uid
filmlist=filmlist.reset_index()
filmlist=filmlist.drop(['index'], 1)
for el in filmlist.element_uid:
    catal.availability[catal.element_uid == el] = str(pd.unique(transactions[transactions.element_uid == el].consumption_mode))
dct=    {  '[S]\nCategories (1, object): [S]': "['subscription']",
         "['purchase', 'rent']": "['purchase', 'rent']",
         '[], Categories (0, object): []': '0',
         '[S, R]\nCategories (2, object): [S, R]': "['rent', 'subscription']",
         "['purchase', 'rent', 'subscription']":"['purchase', 'rent', 'subscription']",
         "['purchase', 'subscription']": "['purchase', 'subscription']",
         "['subscription']": "['subscription']",
         "['purchase']": "['purchase']",
         '[S, R, P]\nCategories (3, object): [S, R, P]': "['purchase', 'rent', 'subscription']",
         '[S, P]\nCategories (2, object): [S, P]':  "['purchase', 'subscription']",
         '[P, R]\nCategories (2, object): [P, R]':"['purchase', 'rent']",
         "['rent']":"['rent']",
         "['rent', 'subscription']": "['rent', 'subscription']",
         '[R, P]\nCategories (2, object): [R, P]': "['purchase', 'rent']",
         '[S, P, R]\nCategories (3, object): [S, P, R]': "['purchase', 'rent', 'subscription']",
         '[P, S]\nCategories (2, object): [P, S]': "['purchase', 'subscription']",
         '[R, S]\nCategories (2, object): [R, S]':"['rent', 'subscription']",
         '[R]\nCategories (1, object): [R]':"['rent']",
         '[P]\nCategories (1, object): [P]': "['purchase']"}
catal.availability=catal.availability.map(dct)
Counter(catal.availability)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Counter({"['subscription']": 507,
         "['purchase', 'rent']": 2660,
         '0': 1479,
         "['rent', 'subscription']": 97,
         "['purchase', 'rent', 'subscription']": 4301,
         "['purchase', 'subscription']": 701,
         "['purchase']": 432,
         "['rent']": 23})

In [5]:

transactions=pd.merge(transactions, catal, on=['element_uid'])

In [6]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)
ratings=ratings.drop(['ts'], 1)
transactions=pd.merge(transactions, ratings, how='left',on=['element_uid', 'user_uid'])

In [7]:
transactions.head()


,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer,attributes,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,type,rating
0,3336,5177,S,4.430518e+07,4282,0,50,"[19924, 28181, 6732, 23032, 270, 24805, 43, 14...","['purchase', 'rent', 'subscription']",90,4.166108e+07,0.739609,45,1.141929,0.654707,movie,9.0
1,3336,349202,S,4.430503e+07,3291,0,50,"[19924, 28181, 6732, 23032, 270, 24805, 43, 14...","['purchase', 'rent', 'subscription']",90,4.166108e+07,0.739609,45,1.141929,0.654707,movie,NaN
2,3336,117582,S,4.430488e+07,5727,3,99,"[19924, 28181, 6732, 23032, 270, 24805, 43, 14...","['purchase', 'rent', 'subscription']",90,4.166108e+07,0.739609,45,1.141929,0.654707,movie,NaN
3,3336,523833,S,4.430484e+07,4346,5,31,"[19924, 28181, 6732, 23032, 270, 24805, 43, 14...","['purchase', 'rent', 'subscription']",90,4.166108e+07,0.739609,45,1.141929,0.654707,movie,NaN
4,3336,571064,S,4.430465e+07,4568,0,99,"[19924, 28181, 6732, 23032, 270, 24805, 43, 14...","['purchase', 'rent', 'subscription']",90,4.166108e+07,0.739609,45,1.141929,0.654707,movie,NaN


In [8]:
elements=pd.get_dummies(transactions.element_uid, sparse = True)

In [9]:
users=pd.get_dummies(transactions.user_uid, sparse = True)

In [10]:
devices=pd.get_dummies(transactions.device_type)
types=pd.get_dummies(transactions.type)
csmode=pd.get_dummies(transactions.consumption_mode)
transactions['duration']=transactions['duration'].map(lambda x: 5 if x == 0 else x)
transactions.watched_time=transactions.watched_time/60
transactions['per']=transactions.watched_time/transactions.duration
transactions=transactions.drop(['device_manufacturer','device_type', 'type', 'consumption_mode'], 1)
transactions=pd.concat([transactions,devices], axis=1)
transactions=pd.concat([transactions,types], axis=1)
transactions=pd.concat([transactions,csmode], axis=1)

In [11]:
transactions["wt_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").watched_time.mean())
transactions["per_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").per.mean())
transactions["wt_element_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").watched_time.mean())
transactions["per_elemnt_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").per.mean())

In [12]:

from collections import Counter
coun = pd.DataFrame.from_dict(Counter(transactions['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='user_uid')


In [13]:
from collections import Counter
coun = pd.DataFrame.from_dict(Counter(transactions['element_uid']), orient='index')
coun['element_uid']=coun.index
coun['quantity_element_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='element_uid')

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

s = transactions['attributes']

mlb = MultiLabelBinarizer(sparse_output=True)

attr=mlb.fit_transform(s)

In [15]:
Counter(transactions.availability)

Counter({"['purchase', 'rent', 'subscription']": 6925311,
         "['purchase', 'rent']": 1134428,
         "['purchase', 'subscription']": 762882,
         "['purchase']": 192344,
         "['subscription']": 592335,
         "['rent', 'subscription']": 31207,
         "['rent']": 4505})

In [16]:
avail=pd.get_dummies(transactions.availability)
transactions=pd.concat([transactions,avail], axis=1)
transactions=transactions.drop(['availability'],1)

In [17]:
transactions['ts-f1']=transactions.ts- transactions.feature_1
transactions.ts=transactions.ts-min(transactions.ts)+1
coef=(max(transactions.ts)-min(transactions.ts))/184
transactions['ts-f1']=transactions['ts-f1']/coef
transactions.ts=transactions.ts/coef
transactions=transactions.drop(['feature_1'],1)

In [18]:
transactions=transactions.drop(['attributes'], 1)
transactions.isnull().sum()/transactions.shape[0]

element_uid                             0.000000
user_uid                                0.000000
ts                                      0.000000
watched_time                            0.000000
duration                                0.000000
feature_2                               0.000000
feature_3                               0.000000
feature_4                               0.000000
feature_5                               0.000000
rating                                  0.962943
per                                     0.000000
0                                       0.000000
1                                       0.000000
2                                       0.000000
3                                       0.000000
4                                       0.000000
5                                       0.000000
6                                       0.000000
movie                                   0.000000
multipart_movie                         0.000000
series              

In [19]:
transactionsM=transactions[transactions.movie == 1]
transactionsMM=transactions[transactions.multipart_movie == 1]
transactionsS=transactions[transactions.series == 1]
transactionsMT=transactionsM[~transactionsM.rating.isnull()]
transactionsMF=transactionsM[transactionsM.rating.isnull()]
transactionsMMT=transactionsMM[~transactionsMM.rating.isnull()]
transactionsMMF=transactionsMM[transactionsMM.rating.isnull()]
transactionsST=transactionsS[~transactionsS.rating.isnull()]
transactionsSF=transactionsS[transactionsS.rating.isnull()]

transactionsMT['interaction']=transactionsMT.rating
transactionsMF['interaction']=transactionsMF.per * 10
transactionsMF['interaction']=transactionsMF['interaction'].map(lambda x: 10 if x>10 else x)


transactionsMMT['interaction']=transactionsMMT.rating
transactionsMMF['interaction']=transactionsMMF.per * 4
transactionsMMF['interaction']=transactionsMMF['interaction'].map(lambda x: 10 if x>10 else x)

transactionsST['interaction']=transactionsST.rating
transactionsSF['interaction']=transactionsSF.per * 1.7
transactionsSF['interaction']=transactionsSF['interaction'].map(lambda x: 10 if x>10 else x)

transactions=transactionsMT.append(transactionsMF)
transactions=transactions.append(transactionsMMT)
transactions=transactions.append(transactionsMMF)
transactions=transactions.append(transactionsST)
transactions=transactions.append(transactionsSF)
transactions=transactions.sort_index()

transactions.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

,element_uid,user_uid,ts,watched_time,duration,feature_2,feature_3,feature_4,feature_5,rating,per,0,1,2,3,4,5,6,movie,multipart_movie,series,P,R,S,wt_user_mean,per_user_mean,wt_element_mean,per_elemnt_mean,quantity_user_views,quantity_element_views,"['purchase', 'rent', 'subscription']","['purchase', 'rent']","['purchase', 'subscription']",['purchase'],"['rent', 'subscription']",['rent'],['subscription'],ts-f1,interaction
0,3336,5177,184.000071,71.366667,90,0.739609,45,1.141929,0.654707,9.0,0.792963,1,0,0,0,0,0,0,1,0,0,0,0,1,76.719048,0.797434,94.06047,1.045116,14,45646,1,0,0,0,0,0,0,188.970839,9.000000
1,3336,349202,183.989107,54.850000,90,0.739609,45,1.141929,0.654707,NaN,0.609444,1,0,0,0,0,0,0,1,0,0,0,0,1,55.497619,0.536494,94.06047,1.045116,7,45646,1,0,0,0,0,0,0,188.959875,6.094444
2,3336,117582,183.978661,95.450000,90,0.739609,45,1.141929,0.654707,NaN,1.060556,0,0,0,1,0,0,0,1,0,0,0,0,1,42.383333,0.405102,94.06047,1.045116,8,45646,1,0,0,0,0,0,0,188.949429,10.000000
3,3336,523833,183.975545,72.433333,90,0.739609,45,1.141929,0.654707,NaN,0.804815,0,0,0,0,0,1,0,1,0,0,0,0,1,62.654762,0.940423,94.06047,1.045116,7,45646,1,0,0,0,0,0,0,188.946313,8.048148
4,3336,571064,183.961773,76.133333,90,0.739609,45,1.141929,0.654707,NaN,0.845926,1,0,0,0,0,0,0,1,0,0,0,0,1,27.601852,0.325679,94.06047,1.045116,9,45646,1,0,0,0,0,0,0,188.932541,8.459259


In [20]:
transactions=transactions.drop(['rating'],1)

In [21]:
inter=transactions[['user_uid','element_uid','interaction']]
inter.head()

,user_uid,element_uid,interaction
0,5177,3336,9.000000
1,349202,3336,6.094444
2,117582,3336,10.000000
3,523833,3336,8.048148
4,571064,3336,8.459259


In [22]:
userfeat=transactions[['user_uid','wt_user_mean','per_user_mean','quantity_user_views']]
elfeat=transactions[['element_uid','wt_element_mean','per_elemnt_mean','quantity_element_views', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'movie','multipart_movie','series', 'duration']]


In [23]:
userfeat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9643012 entries, 0 to 9643011
Data columns (total 4 columns):
user_uid               uint32
wt_user_mean           float64
per_user_mean          float64
quantity_user_views    int64
dtypes: float64(2), int64(1), uint32(1)
memory usage: 331.1 MB


In [24]:
elfeat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9643012 entries, 0 to 9643011
Data columns (total 12 columns):
element_uid               uint16
wt_element_mean           float64
per_elemnt_mean           float64
quantity_element_views    int64
feature_2                 float64
feature_3                 int64
feature_4                 float64
feature_5                 float64
movie                     uint8
multipart_movie           uint8
series                    uint8
duration                  int64
dtypes: float64(5), int64(3), uint16(1), uint8(3)
memory usage: 708.1 MB


In [25]:
from lightfm.data import Dataset

dataset=Dataset()
#dataset.fit(transactions['user_uid'],transactions['element_uid'], user_features=['wt_user_mean','per_user_mean','quantity_user_views'], item_features=['wt_element_mean','per_elemnt_mean','quantity_element_views', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'movie','multipart_movie','series', 'duration'])


In [26]:
transactions.head()

,element_uid,user_uid,ts,watched_time,duration,feature_2,feature_3,feature_4,feature_5,per,0,1,2,3,4,5,6,movie,multipart_movie,series,P,R,S,wt_user_mean,per_user_mean,wt_element_mean,per_elemnt_mean,quantity_user_views,quantity_element_views,"['purchase', 'rent', 'subscription']","['purchase', 'rent']","['purchase', 'subscription']",['purchase'],"['rent', 'subscription']",['rent'],['subscription'],ts-f1,interaction
0,3336,5177,184.000071,71.366667,90,0.739609,45,1.141929,0.654707,0.792963,1,0,0,0,0,0,0,1,0,0,0,0,1,76.719048,0.797434,94.06047,1.045116,14,45646,1,0,0,0,0,0,0,188.970839,9.000000
1,3336,349202,183.989107,54.850000,90,0.739609,45,1.141929,0.654707,0.609444,1,0,0,0,0,0,0,1,0,0,0,0,1,55.497619,0.536494,94.06047,1.045116,7,45646,1,0,0,0,0,0,0,188.959875,6.094444
2,3336,117582,183.978661,95.450000,90,0.739609,45,1.141929,0.654707,1.060556,0,0,0,1,0,0,0,1,0,0,0,0,1,42.383333,0.405102,94.06047,1.045116,8,45646,1,0,0,0,0,0,0,188.949429,10.000000
3,3336,523833,183.975545,72.433333,90,0.739609,45,1.141929,0.654707,0.804815,0,0,0,0,0,1,0,1,0,0,0,0,1,62.654762,0.940423,94.06047,1.045116,7,45646,1,0,0,0,0,0,0,188.946313,8.048148
4,3336,571064,183.961773,76.133333,90,0.739609,45,1.141929,0.654707,0.845926,1,0,0,0,0,0,0,1,0,0,0,0,1,27.601852,0.325679,94.06047,1.045116,9,45646,1,0,0,0,0,0,0,188.932541,8.459259


In [27]:
dataset.fit(users = (row['user_uid'] for index, row in transactions.iterrows()),
            items= (row['element_uid'] for index, row in transactions.iterrows()),
            user_features= ((row['wt_user_mean'],row['per_user_mean'],row['quantity_user_views']) for index, row in transactions.iterrows()),
            item_features= ((row['wt_element_mean'],row['per_elemnt_mean'], row['quantity_element_views'], row['feature_2'], row['feature_3'],row[ 'feature_4'],row['feature_5'],row['movie'],row['multipart_movie'],row['series'], row['duration']) for index, row in transactions.iterrows())
            )


In [34]:
ufe=dataset.build_user_features((row['user_uid'], [row['wt_user_mean'],row['per_user_mean'],row['quantity_user_views']]) for index, row in transactions.iterrows())

ValueError: Feature 76.71904761904761 not in eature mapping. Call fit first.

In [28]:
features = ((row['element_uid'],[row['wt_element_mean'],row['per_elemnt_mean'], row['quantity_element_views'], row['feature_2'], row['feature_3'],row[ 'feature_4'],row['feature_5'],row['movie'],row['multipart_movie'],row['series'], row['duration']]) for index, row in elfeat.iterrows())
ife=dataset.build_item_features(features)

ValueError: Feature 94.06046970161714 not in eature mapping. Call fit first.

In [32]:
(interactions, weights)=dataset.build_interactions(inter.values)

In [47]:
n_users,n_items= interactions.shape

In [52]:
interactions.shape

(499663, 8296)

In [48]:
from lightfm import LightFM

model = LightFM(loss='warp')
model.fit(interactions)

In [ ]:
answ={}
#for user_id in test_users:
scores=model.predict(1310342373, np.arange(n_items))
top_items= transactions['element_uid'][np.argsort(-scores)]
dct0={'{}'.format(131072) : top_items[:20]}
z=answ.update(dct0)

In [76]:
test_users

{0,
 131072,
 2,
 262145,
 524291,
 131077,
 6,
 393222,
 393223,
 262154,
 131084,
 13,
 393228,
 16,
 524306,
 262164,
 25,
 393242,
 30,
 393248,
 131105,
 262179,
 39,
 131112,
 47,
 131119,
 131121,
 524335,
 51,
 393270,
 61,
 64,
 262208,
 66,
 262211,
 524353,
 72,
 262216,
 131146,
 262222,
 131151,
 393296,
 91,
 131165,
 524381,
 97,
 524386,
 100,
 393318,
 393325,
 131186,
 131188,
 262260,
 120,
 121,
 393340,
 524419,
 134,
 524424,
 131210,
 262282,
 393355,
 262287,
 131216,
 393364,
 262299,
 156,
 131228,
 131230,
 262300,
 161,
 393381,
 262312,
 131245,
 181,
 262326,
 524476,
 262337,
 393411,
 524483,
 262346,
 524495,
 262352,
 210,
 262355,
 262359,
 393432,
 262361,
 393434,
 131293,
 393446,
 524524,
 524525,
 131316,
 393465,
 524540,
 262397,
 131330,
 259,
 524548,
 262407,
 524554,
 271,
 524565,
 262422,
 262423,
 524567,
 524576,
 291,
 131363,
 293,
 131364,
 295,
 131367,
 393512,
 262442,
 262443,
 524584,
 524593,
 524597,
 131385,
 262457,
 131388,

In [79]:
answ

{'131072': 81     3336
 49     3336
 221    3336
 43     3336
 83     3336
 452    3336
 525    3336
 14     3336
 15     3336
 12     3336
 116    3336
 227    3336
 6      3336
 320    3336
 86     3336
 533    3336
 91     3336
 82     3336
 225    3336
 0      3336
 Name: element_uid, dtype: uint16}

In [55]:
d=[]
for user in test_users:
    if user not in pd.unique(transactions.user_uid):
        d=d.append(user)

KeyboardInterrupt: 

In [56]:
d=[396764,
275799,
180884,
328547,
592316,
202305,
353063,
247919]
for x in d:
    test_users.remove(x)

In [59]:
524292 -499663

24629